In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep, time
from random import uniform

In [2]:
input_file = "input.txt"

if input_file:
    # read keywords from file
    keywords = []
    with open(input_file, "r") as f:
        for line in f:
            tokens = line.split('*')
            if len(tokens) == 3:
                keywords.append(tokens[1])
else:
    # use default keywords
    keywords = ['subreddit', 'wikipedia']    
                
print(keywords)

['goldfish', 'context truncation', 'conversationalist', 'shared interests', 'TV shows', 'all-vs-all', 'natural dialog', 'co-rating', 'BlenderBot BPE', 'overexpressed', 'conversational attributes', 'long-context', 'granularity', 'joint goal accuracy', 'PrLM', 'special token', 'two-way classification', 'binary classification', 'auxilary relationship', 'auxilary edge', 'auxilary node', 'gate control GNN', 'gate fusion GNN', 'DST perspective', 'noise-free', 'indicator token', 'copy mechanism', '10 different random seeds', '5 different random seeds', 'confuse the model', 'core components', 'taxi', 'bias mitigation', 'adequacy', 'focal loss', 'statistical artifacts', 'multitask-setting', 'multi-task setting', 'reversed', 'partial input bias', 'decent prediction', 'preserve the meaning']


In [3]:
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

# the returned number is unstable, need multiple clicks
clicks = 5

# the wait time needs to be randomized so the scraper is not identified as a bot
# the range is uniformly sampled between [a, b] seconds
# the time also needs to be large enough so the search results could be returned
a = 1
b = 1.5

# the output file, which will be manually edited after obtaining all output
fo = open("output.txt", "a")

# with the current parameters, about 20 seconds per keyword
for keyword in keywords:
    
    start = time()
    
    # random sleep for a few seconds
    sleep(uniform(5, 10))

    url = f"https://aclanthology.org/search/?q={keyword.replace(' ', '+')}"

    # start the driver
    driver.get(url)
    
    nums = []

    # switch between the options multiple times
    for _ in range(clicks):
        
        # expand the dropdown menu
        buttons = driver.find_elements(by=By.CLASS_NAME, value="gsc-selected-option-container")
        buttons[0].click()
    
        # choose year of publication
        buttons =  driver.find_elements(by=By.CLASS_NAME, value="gsc-option-menu-item")
        buttons[1].click()

        # some time is needed for the result to change
        sleep(uniform(a,b))

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # result in the format "About 31 results (0.11 seconds)"
        result = soup.find_all("div", {"class": "gsc-result-info"})

        # get the number
        try:
            num = result[0].string.split(' ')[1].replace(',', '')
        except:
            num = 0
        nums.append(num)
        
        # expand the dropdown menu
        buttons = driver.find_elements(by=By.CLASS_NAME, value="gsc-selected-option-container")
        buttons[0].click()
        
        # change back to relevance
        buttons =  driver.find_elements(by=By.CLASS_NAME, value="gsc-option-menu-item")
        buttons[0].click()
        sleep(uniform(a,b))
        
    end = time()
        
    print(keyword, min(nums), f"({end-start:.2f} seconds)")
    
    fo.write(f"{keyword}, {min(nums)}\n")
    fo.flush()

# quit the driver
driver.quit()

# close the file
fo.close()

goldfish 2 (24.81 seconds)
context truncation 24 (23.81 seconds)
conversationalist 2 (21.15 seconds)
shared interests 165 (23.33 seconds)
TV shows 67 (19.34 seconds)
all-vs-all 947 (23.18 seconds)
natural dialog 1720 (21.92 seconds)
co-rating 7 (21.25 seconds)
BlenderBot BPE 0 (21.40 seconds)
overexpressed 0 (21.67 seconds)
conversational attributes 51 (24.71 seconds)
long-context 1040 (21.26 seconds)
granularity 268 (20.04 seconds)
joint goal accuracy 60 (19.91 seconds)
PrLM 5 (20.23 seconds)
special token 69 (21.80 seconds)
two-way classification 112 (21.63 seconds)
binary classification 306 (23.42 seconds)
auxilary relationship 55 (22.76 seconds)
auxilary edge 7 (20.42 seconds)
auxilary node 0 (20.58 seconds)
gate control GNN 0 (20.97 seconds)
gate fusion GNN 0 (19.28 seconds)
DST perspective 3 (22.26 seconds)
noise-free 21 (21.93 seconds)
indicator token 9 (22.52 seconds)
copy mechanism 1480 (19.87 seconds)
10 different random seeds 20 (22.86 seconds)
5 different random seeds 0 (20